# Analyze classified comments

After the first part of the project, to find suitable categories and classify all comments according to the found cateogries, we can start analyzing the classified comments.


In [61]:
import pickle

import data
import prompt
from dotenv import load_dotenv

import numpy as np
import pandas as pd
import openai
import matplotlib.pyplot as plt

In [ ]:
load_dotenv()

In [ ]:
df = data.load_data("Umfrage DE submissions 2024-07-31 00_25.csv")

In [4]:
df = df[df.Stadtteil != "SPAM"]
df = df[(df["Positive"].str.len() > 7) | (df["Negative"].str.len() > 7)]

In [5]:
client = openai.Client()

## Process categorized comments


Actually, before we can analyze the comments, we need to process them and merge them with the original dataset so that each survey answer has the detected categories.


In [6]:
with open("artifacts/pos_results.pkl", "rb") as f:
    pos_comments = pickle.load(f)

with open("artifacts/neg_results.pkl", "rb") as f:
    neg_comments = pickle.load(f)

In [7]:
pos_comments = [c if c.startswith("[") else "[" + c + "]" for _, c in pos_comments]
neg_comments = [c if c.startswith("[") else "[" + c + "]" for _, c in neg_comments]

In [8]:
pos_comments = [c if c != "[]" else "['UNCLEAR']" for c in pos_comments]
neg_comments = [c if c != "[]" else "['UNCLEAR']" for c in neg_comments]

In [ ]:
pos_comments[:10]

In [10]:
pos_comments_list = []
for comment in pos_comments:
    cats = []
    if "UNCLEAR" in comment:
        cats.append("UNCLEAR")
    else:
        for cat in data.final_categories:
            if cat in comment:
                cats.append(cat)
    pos_comments_list.append(cats)

neg_comments_list = []
for comment in neg_comments:
    cats = []
    if "UNCLEAR" in comment:
        cats.append("UNCLEAR")
    else:
        for cat in data.final_categories:
            if cat in comment:
                cats.append(cat)
    neg_comments_list.append(cats)

In [ ]:
len(pos_comments_list), len(neg_comments_list)

In [ ]:
for comment, cat in zip(df.Positive.head(20), pos_comments_list[:20]):
    print("Comment:", comment)
    print("Categories:", ",".join(cat))
    print()

In [13]:
df_positive = df[["Date", "Stadtteil", "Region", "Positive"]]
df_negative = df[["Date", "Stadtteil", "Region", "Negative"]]

In [14]:
df_positive["category"] = [
    list(
        map(
            lambda x: data.category_mapping.get(x) if x in data.category_mapping else x,
            l,
        )
    )
    for l in pos_comments_list
]
df_positive["no_categories"] = df_positive.category.apply(len)
# df_positive["spam"] = df_positive["category"].apply(lambda x: "SPAM" in x)
df_positive["unclear"] = df_positive["category"].apply(lambda x: "UNCLEAR" in x)

df_negative["category"] = [
    list(
        map(
            lambda x: data.category_mapping.get(x) if x in data.category_mapping else x,
            l,
        )
    )
    for l in neg_comments_list
]
df_negative["no_categories"] = df_negative.category.apply(len)
# df_negative["spam"] = df_negative["category"].apply(lambda x: "SPAM" in x)
df_negative["unclear"] = df_negative["category"].apply(lambda x: "UNCLEAR" in x)

In [ ]:
df_positive.head()

In [ ]:
df_negative.head()

## First look at comments


In [ ]:
# df_positive.spam.sum() / len(df_positive),
df_positive.unclear.sum() / len(df_positive)

In [ ]:
# df_negative.spam.sum() / len(df_positive),
df_negative.unclear.sum() / len(df_negative)

In [ ]:
for x in df_positive[df_positive.unclear][["Positive", "category"]].itertuples(
    index=False
):
    print(x.Positive)

In [ ]:
for x in df_negative[df_negative.unclear][["Negative", "category"]].itertuples(
    index=False
):
    print(x.Negative)

In [ ]:
df_positive.no_categories.value_counts()

In [ ]:
df_negative.no_categories.value_counts()

In [ ]:
for x in df_positive[df_positive.no_categories >= 4].Positive:
    print(x)
    print()

In [ ]:
for x in df_negative[df_negative.no_categories >= 4].Negative:
    print(x)
    print()

## Visualizations


In [ ]:
pure_cats = df_positive.category.explode()
pure_cats = pure_cats.loc[lambda x: x != "UNCLEAR"]
pure_cats.value_counts()

In [ ]:
pure_cats = df_positive.category.explode()
pure_cats = pure_cats.loc[lambda x: x != "UNCLEAR"]
pure_cats.value_counts()

ax = pure_cats.value_counts(ascending=True).plot(
    kind="barh", width=0.8, color="green", legend=False, alpha=0.8, edgecolor="k"
)
plt.title("Positive Aspekte über das Familienleben mit Baby in Frankfurt")
plt.ylabel("")
plt.xlim((0, 450))
ax.bar_label(ax.containers[0], padding=3)
plt.savefig("output/Positive_Aspekte_gesamt.png", bbox_inches="tight")
plt.savefig("output/Positive_Aspekte_gesamt.pdf", bbox_inches="tight")

In [ ]:
pure_cats = df_negative.category.explode()
pure_cats = pure_cats.loc[lambda x: x != "UNCLEAR"]
pure_cats.value_counts()

In [ ]:
pure_cats = df_negative.category.explode()
pure_cats = pure_cats.loc[lambda x: x != "UNCLEAR"]
pure_cats.value_counts()

ax = pure_cats.value_counts(ascending=True).plot(
    kind="barh", width=0.8, color="darkred", legend=False, alpha=0.8, edgecolor="k"
)

plt.title("Negative Aspekte über das Familienleben mit Baby in Frankfurt")
plt.ylabel("")
plt.xlim((0, 320))
ax.bar_label(ax.containers[0], padding=3)
plt.savefig("output/Negative_Aspekte_gesamt.png", bbox_inches="tight")
plt.savefig("output/Negative_Aspekte_gesamt.pdf", bbox_inches="tight")

### Regional analysis


In [ ]:
for region in df.Region.dropna().unique():
    for cat, df_reg in [
        ("positive", df_positive.query("Region == @region")),
        ("negative", df_negative.query("Region == @region")),
    ]:
        plt.figure(figsize=(10, 6))
        pure_cats = df_reg.category.explode()
        pure_cats = pure_cats.loc[lambda x: x != "UNCLEAR"]
        ax = pure_cats.value_counts(ascending=True).plot(
            kind="barh",
            width=0.8,
            color="darkred" if cat == "negative" else "green",
            legend=False,
        )
        # plt.title(f"Categories of {cat} comments about living in Frankfurt {region} with a baby")
        plt.title(
            f"{cat.title()} Aspekte über das Familienleben mit Baby in Frankfurt {region}"
        )
        plt.ylabel("")
        plt.xlim((0, pure_cats.value_counts().values[0] + 10))
        ax.bar_label(ax.containers[0], padding=3)
        plt.savefig(f"output/{region}_{cat}.png", bbox_inches="tight")

In [ ]:
df_positive

In [ ]:
df_positive.explode("category").query("category != 'UNCLEAR'").groupby(
    ["Region", "category"]
).count().groupby(level=0, group_keys=False).apply(
    lambda x: x.sort_values("Positive", ascending=False).head(3)
)[["Positive"]]

In [ ]:
df_positive.explode("category").query("category != 'UNCLEAR'").groupby(
    ["Region", "category"]
).count().groupby(level=0, group_keys=False).apply(
    lambda x: x.sort_values("Positive", ascending=False).head(3)
)[["Positive"]].plot(kind="bar")

In [ ]:
df_negative.explode("category").query("category != 'UNCLEAR'").groupby(
    ["Region", "category"]
).count().groupby(level=0, group_keys=False).apply(
    lambda x: x.sort_values("Negative", ascending=False).head(3)
)[["Negative"]]

## Analyze categories


In [ ]:
df_negative[
    df_negative.category.apply(
        lambda x: data.category_mapping["Childcare and Early Education Resources"] in x
    )
].Negative

In [ ]:
# Warning! This cell will cost money when using OpenAI's API
model = "gpt-4o-mini"
temperature = 0

pure_cats = df_negative.category.explode()
pure_cats = pure_cats.loc[lambda x: x != "UNCLEAR"]
pure_cats = pure_cats.value_counts()
for cat in pure_cats.index:
    comments = df_negative[
        (df_negative.category.apply(lambda x: cat in x))
        & (df_negative.no_categories == 1)
    ].Negative

    response = client.chat.completions.create(
        model=model,
        # response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": prompt.SYSTEM_PROMPT_SUMMARIZE_TOPICS_PER_CAT,
            },
            {
                "role": "user",
                "content": prompt.USER_PROMPT_SUMMARIZE_TOPICS_PER_CAT.format(
                    comments="\n".join(
                        f"{i}. {comment}" for i, comment in enumerate(comments)
                    ),
                ),
            },
        ],
        temperature=temperature,
    )

    print(response.choices[0].finish_reason)
    print("Category: ", cat)
    print(response.choices[0].message.content)

## Antworten nach Kategorien


In [47]:
df.to_excel("output/Antworten_mit_Regionen.xlsx")

In [48]:
# store your dataframes in a dict, where the key is the sheet name you want
frames = {"Positive": df_positive, "Negative": df_negative}

with pd.ExcelWriter(
    "output/Antworten_mit_Kategorien.xlsx", engine="xlsxwriter"
) as writer:
    # now loop thru and put each on a specific sheet
    for sheet, frame in frames.items():  # .use .items for python 3.X
        frame.to_excel(writer, sheet_name=sheet)

In [ ]:
df.Stadtteil.unique()

In [66]:
city_parts = []
categories = []
sentiments = []
values = []


for city_part in df.Stadtteil.unique():
    for cat in data.final_categories_de:
        for sentiment in ["positive", "negative"]:
            city_parts.append(city_part)
            categories.append(cat)
            sentiments.append(sentiment)
            if sentiment == "positive":
                df_ = df_positive
            else:
                df_ = df_negative
                
            values.append(
                df_[
                    df_positive.Stadtteil == city_part
                ].category.explode().value_counts().get(cat, 0)
            )

In [ ]:
df_pos_neg_cat_per_city_part = pd.DataFrame({'Stadtteil': city_parts, 'Kategorie': categories, 'Sentiment': sentiments, 'Anzahl': values})
df_pos_neg_cat_per_city_part

In [ ]:
df_pos_neg_cat_per_city_part.pivot(index='Stadtteil', columns=['Kategorie', 'Sentiment'], values='Anzahl').fillna(0)

In [72]:
df_pos_neg_cat_per_city_part.pivot(index='Stadtteil', columns=['Kategorie', 'Sentiment'], values='Anzahl').fillna(0).to_excel("output/Pos_Neg_Kategorien_pro_Stadtteil_wide.xlsx")

In [70]:
df_pos_neg_cat_per_city_part.to_excel(
    "output/Pos_Neg_Kategorien_pro_Stadtteil_tidy.xlsx"
)